In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from datetime import datetime 
import time

import warnings
warnings.filterwarnings('ignore',category=FutureWarning)

from sklearn.metrics import max_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
#from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf
from tensorflow import keras

from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential, load_model

In [2]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]

In [3]:
tf.config.experimental.list_physical_devices(device_type = None)

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'),
 PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU'),
 PhysicalDevice(name='/physical_device:XLA_GPU:1', device_type='XLA_GPU')]

In [4]:
f16 = pd.read_csv('data/f16_l3.csv')

In [5]:
X = f16['Force']
y = f16['Acceleration3']

In [6]:
#X = MinMaxScaler().fit_transform(X.values.reshape(-1,1))

In [7]:
#Declare the number of model iterations we will run 
test_iterations = 4

#Declare the size of the initial train test split
test_size = 0.20

#Declare the number of hidden layers to add
layers = 4

#Declare the density of the hidden layers
density = 128

#Declare the type of activation for the hidden layers
activation = 'relu' 
#activation = ks.layers.LeakyReLU(alpha = 0.01)

#Assign the loss function the model will use to train
loss = 'mean_squared_error'

#Assign the optimizer for the compilitation
optimizer = 'adam'

#Declare the patience count used for early stopping
patience = 25

#Declare the test size used for early stopping train test split
es_test_size = 0.25

#Declare the batch size for use in the model
batch_size = 64

#Declare the maximum number of epochs for our model
epochs = 250

In [8]:
X_train, X_true, y_train, y_true = train_test_split(
    X, y, random_state = 8669, test_size = test_size)

In [9]:
#Assigns the computation to be performed via GPU Device:0
with tf.device('/gpu:0'):
    model = Sequential()

    #Creates a for loop that will add the number of layers based on the variable declared in the previous cell    
    for i in range(layers):
        #Adds a model layer with density and activation based on the variables declared in the previous cell
        model.add(Dense(density, activation = activation))
    
    model.add(Dense(1, activation='linear'))
    
    #model.compile(loss = loss, optimizer = optimizer, metrics = ['mse'])

In [10]:
es = EarlyStopping(monitor = 'loss', patience = patience, restore_best_weights = True)
mc = ModelCheckpoint(filepath = 'test_model.h5', monitor = 'loss', save_best_only=True)
X_es_train, X_es_true, y_es_train, y_es_true = train_test_split(X_train, y_train, test_size = es_test_size, random_state = 8669)

In [11]:
#Creates a dataframe by which we will eventually put in our list created above
model_record = pd.DataFrame(columns = ['model_num', 'loss_type', 'time', 'epochs', 'r2', 'mae', 'mse', 'rmse', 'max_error'])

#Creates a dataframe by which our model's predicted values and true values will be stored
predict_record = pd.DataFrame(y_true).reset_index(drop = True)

#Creates a numpy array by which the for loop will use to count model runs and is then used to name df columns
model_counter = np.array([0])

In [12]:
#Assigns the computation to be performed via GPU Device:0
with tf.device('/gpu:0'):
    
#Performs model training repeatedly based on the variable declared previously    
    for i in range(test_iterations):
        model.compile(loss = loss, optimizer = optimizer, metrics = ['mse'])

        #Creates an empty list for storing model metrics and other information 
        record_list = list() 
        #Starts a counter that adds 1 everytime a fitting is performed
        model_counter = model_counter + 1
        #Starts a timer to end after the round of fitting is complete
        start_time = datetime.now()
        
        #Fits our model/batch_size and epochs are declared previously
        #model.fit(x = X_train, y = y_train.values, 
        #          batch_size = batch_size, epochs = epochs)
        
        history = model.fit(x = X_es_train, y = y_es_train.values, 
                  validation_data = (X_es_true, y_es_true),
                  batch_size = batch_size, epochs = epochs, 
                  callbacks = [es, mc])
        
        #Saves our model predictions
        y_pred = model.predict(X_true)
        
        #Saves our model run #, time to run, and model metrics to our temporary list
        record_list.extend([len(model_record)+1, #A model iteration count
                            loss, #Records the loss metric used to fit the model
                            format(datetime.now() - start_time), #Calculates the time it took to complete the model
                            len(history.history['loss']), #The epoch number of our model checkpoint 
                            r2_score(y_true, y_pred), #R2
                            mean_absolute_error(y_true, y_pred), #MAE
                            mean_squared_error(y_true, y_pred), #MSE
                            np.sqrt(mean_squared_error(y_true, y_pred)), #RMSE
                            max_error(y_true, y_pred) #MaxError
                           ])        
        
        #Adds the temporary list of model metrics (etc) to the end of a dataframe
        model_record.loc[len(model_record)] = record_list
        
        #Converts our predictions to a dataframe so it will play nice
        y_pred_df = pd.DataFrame(y_pred)
        
        #Adds predictions as a column to the end of a dataframe and names is accordingly
        predict_record = pd.concat([predict_record, y_pred_df], axis = 1)
        predict_record = predict_record.rename(columns = {0 : 'm' + str(model_counter[0])})
        
        #Calculates the residual values for each prediction and stores it as a dataframe 
        residuals_df = pd.DataFrame(abs(predict_record.iloc[:,len(predict_record.columns)-1] - predict_record.iloc[:,0]))
        
        #Adds y residuals as a column to the end of a dataframe and names is accordingly
        predict_record = pd.concat([predict_record, residuals_df], axis = 1)
        predict_record = predict_record.rename(columns = {0 : 'res' + str(model_counter[0])})

Epoch 1/250
1006/1006 [==============================] - 4s 3ms/step - loss: 0.2328 - mse: 0.2328 - val_loss: 0.2385 - val_mse: 0.2385
Epoch 2/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2286 - mse: 0.2286 - val_loss: 0.2382 - val_mse: 0.2382
Epoch 3/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2276 - mse: 0.2276 - val_loss: 0.2378 - val_mse: 0.2378
Epoch 4/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2266 - mse: 0.2266 - val_loss: 0.2343 - val_mse: 0.2343
Epoch 5/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2249 - mse: 0.2249 - val_loss: 0.2338 - val_mse: 0.2338
Epoch 6/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2239 - mse: 0.2239 - val_loss: 0.2387 - val_mse: 0.2387
Epoch 7/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2214 - mse: 0.2214 - val_loss: 0.2301 - val_mse: 0.2301
Epoch 8/250
1006/1006 [==============================] 

1006/1006 [==============================] - 3s 3ms/step - loss: 0.2095 - mse: 0.2095 - val_loss: 0.2178 - val_mse: 0.2178
Epoch 62/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2091 - mse: 0.2091 - val_loss: 0.2181 - val_mse: 0.2181
Epoch 63/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2094 - mse: 0.2094 - val_loss: 0.2187 - val_mse: 0.2187
Epoch 64/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2092 - mse: 0.2092 - val_loss: 0.2186 - val_mse: 0.2186
Epoch 65/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2088 - mse: 0.2088 - val_loss: 0.2176 - val_mse: 0.2176
Epoch 66/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2095 - mse: 0.2095 - val_loss: 0.2185 - val_mse: 0.2185
Epoch 67/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2101 - mse: 0.2101 - val_loss: 0.2195 - val_mse: 0.2195
Epoch 68/250
1006/1006 [==============================] - 3s 

1006/1006 [==============================] - 3s 3ms/step - loss: 0.2086 - mse: 0.2086 - val_loss: 0.2187 - val_mse: 0.2187
Epoch 122/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2086 - mse: 0.2086 - val_loss: 0.2169 - val_mse: 0.2169
Epoch 123/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2088 - mse: 0.2088 - val_loss: 0.2187 - val_mse: 0.2187
Epoch 124/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2087 - mse: 0.2087 - val_loss: 0.2181 - val_mse: 0.2181
Epoch 125/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2086 - mse: 0.2086 - val_loss: 0.2176 - val_mse: 0.2176
Epoch 126/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2084 - mse: 0.2084 - val_loss: 0.2203 - val_mse: 0.2203
Epoch 127/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2088 - mse: 0.2088 - val_loss: 0.2172 - val_mse: 0.2172
Epoch 128/250
1006/1006 [==============================

Epoch 2/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2083 - mse: 0.2083 - val_loss: 0.2168 - val_mse: 0.2168
Epoch 3/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2083 - mse: 0.2083 - val_loss: 0.2187 - val_mse: 0.2187
Epoch 4/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2084 - mse: 0.2084 - val_loss: 0.2172 - val_mse: 0.2172
Epoch 5/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2084 - mse: 0.2084 - val_loss: 0.2172 - val_mse: 0.2172
Epoch 6/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2086 - mse: 0.2086 - val_loss: 0.2183 - val_mse: 0.2183
Epoch 7/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2082 - mse: 0.2082 - val_loss: 0.2174 - val_mse: 0.2174
Epoch 8/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2083 - mse: 0.2083 - val_loss: 0.2178 - val_mse: 0.2178
Epoch 9/250
1006/1006 [==============================] 

1006/1006 [==============================] - 3s 3ms/step - loss: 0.2083 - mse: 0.2083 - val_loss: 0.2177 - val_mse: 0.2177
Epoch 17/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2082 - mse: 0.2082 - val_loss: 0.2176 - val_mse: 0.2176
Epoch 18/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2080 - mse: 0.2080 - val_loss: 0.2183 - val_mse: 0.2183
Epoch 19/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2085 - mse: 0.2085 - val_loss: 0.2174 - val_mse: 0.2174
Epoch 20/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2081 - mse: 0.2081 - val_loss: 0.2170 - val_mse: 0.2170
Epoch 21/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2080 - mse: 0.2080 - val_loss: 0.2178 - val_mse: 0.2178
Epoch 22/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2081 - mse: 0.2081 - val_loss: 0.2199 - val_mse: 0.2199
Epoch 23/250
1006/1006 [==============================] - 3s 

1006/1006 [==============================] - 3s 3ms/step - loss: 0.2083 - mse: 0.2083 - val_loss: 0.2173 - val_mse: 0.2173
Epoch 42/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2079 - mse: 0.2079 - val_loss: 0.2177 - val_mse: 0.2177
Epoch 43/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2080 - mse: 0.2080 - val_loss: 0.2173 - val_mse: 0.2173
Epoch 44/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2083 - mse: 0.2083 - val_loss: 0.2170 - val_mse: 0.2170
Epoch 45/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2082 - mse: 0.2082 - val_loss: 0.2189 - val_mse: 0.2189
Epoch 46/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2080 - mse: 0.2080 - val_loss: 0.2196 - val_mse: 0.2196
Epoch 47/250
1006/1006 [==============================] - 3s 3ms/step - loss: 0.2081 - mse: 0.2081 - val_loss: 0.2167 - val_mse: 0.2167
Epoch 48/250
1006/1006 [==============================] - 3s 

In [13]:
model_record

,model_num,loss_type,time,epochs,r2,mae,mse,rmse,max_error
0,1,mean_squared_error,0:09:10.544527,179,0.186899,0.229131,0.217333,0.466190,5.226147
1,2,mean_squared_error,0:02:21.821995,46,0.189253,0.229513,0.216704,0.465514,5.231953
2,3,mean_squared_error,0:01:48.031540,35,0.188763,0.228867,0.216835,0.465655,5.231925
3,4,mean_squared_error,0:02:46.536850,54,0.192455,0.230446,0.215848,0.464594,5.231256


In [14]:
predict_record

,Acceleration3,m1,res1,m2,res2,m3,res3,m4,res4
0,-0.314890,-0.098931,0.215959,-0.075298,0.239592,-0.053967,0.260923,-0.105321,0.209569
1,-0.267150,-0.276631,0.009481,-0.281761,0.014611,-0.272139,0.004989,-0.295437,0.028287
2,0.257710,0.254854,0.002856,0.246601,0.011109,0.280225,0.022515,0.323467,0.065757
3,-0.193330,-0.099771,0.093559,-0.107190,0.086140,-0.106385,0.086945,-0.105088,0.088242
4,-0.409560,-0.120076,0.289484,-0.122826,0.286734,-0.120458,0.289102,-0.135839,0.273721
...,...,...,...,...,...,...,...,...,...
21446,-0.021237,-0.086881,0.065644,-0.097871,0.076634,-0.094188,0.072951,-0.104534,0.083297
21447,-0.264320,0.098779,0.363099,0.086443,0.350763,0.097440,0.361760,0.074308,0.338628
21448,0.060178,-0.068791,0.128969,-0.064325,0.124503,-0.038473,0.098651,-0.106361,0.166539
21449,0.535320,0.240120,0.295200,0.210879,0.324441,0.241959,0.293361,0.277449,0.257871


In [15]:
#Nifty call to confirm our variables were properly inputted into our model
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               256       
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_2 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_3 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 129       
Total params: 49,921
Trainable params: 49,921
Non-trainable params: 0
_________________________________________________________________


In [16]:
#model_record.to_csv('data/rnn_results_1.csv')
#predict_record.to_csv('data/rnn_predictions_1.csv')